# Portfolio management

*NB*: This notebook takes at its input CSV files from the Ken French data library, which we have uploaded on the hub. You should save these files in the relative path '../data/famafrench/'. You also need a code snipped called 'return_binscatter.py' which we've uploaded, saved in the same folder as this notebook.

In [1]:
import pandas as pd

In [2]:
# get data on market excess return
mkt = pd.read_csv('../data/famafrench/F-F_Research_Data_Factors.CSV', skiprows = 3, nrows = 1112)
mkt.head()

,Unnamed: 0,Mkt-RF,SMB,HML,RF
0,192607,2.96,-2.30,-2.87,0.22
1,192608,2.64,-1.40,4.19,0.25
2,192609,0.36,-1.32,0.01,0.23
3,192610,-3.24,0.04,0.51,0.32
4,192611,2.53,-0.20,-0.35,0.31


In [3]:
# cleaning
mkt = mkt.rename(columns = {'Unnamed: 0':'Date'})
mkt = mkt.set_index(pd.to_datetime(mkt['Date'],format='%Y%m'))
mkt = mkt[['Mkt-RF','RF']]
mkt.head()

,Mkt-RF,RF
Date,,
1926-07-01,2.96,0.22
1926-08-01,2.64,0.25
1926-09-01,0.36,0.23
1926-10-01,-3.24,0.32
1926-11-01,2.53,0.31


In [4]:
# get data on industry excess returns
ind = pd.read_csv('../data/famafrench/10_Industry_Portfolios.CSV',skiprows = 11,nrows=1112)
ind = ind.rename(columns = {'Unnamed: 0':'Date'})
ind = ind.set_index(pd.to_datetime(ind['Date'],format='%Y%m')).drop(columns='Date')
ind.head()

,NoDur,Durbl,Manuf,Enrgy,HiTec,Telcm,Shops,Hlth,Utils,Other
Date,,,,,,,,,,
1926-07-01,1.45,15.55,4.69,-1.18,2.90,0.83,0.11,1.77,7.04,2.16
1926-08-01,3.97,3.68,2.81,3.47,2.66,2.17,-0.71,4.25,-1.69,4.38
1926-09-01,1.14,4.80,1.15,-3.39,-0.38,2.41,0.21,0.69,2.04,0.29
1926-10-01,-1.24,-8.23,-3.63,-0.78,-4.58,-0.11,-2.29,-0.57,-2.63,-2.85
1926-11-01,5.21,-0.19,4.10,0.01,4.71,1.63,6.43,5.42,3.71,2.11


In [5]:
# join
df = mkt.join(ind)
df.head()

,Mkt-RF,RF,NoDur,Durbl,Manuf,Enrgy,HiTec,Telcm,Shops,Hlth,Utils,Other
Date,,,,,,,,,,,,
1926-07-01,2.96,0.22,1.45,15.55,4.69,-1.18,2.90,0.83,0.11,1.77,7.04,2.16
1926-08-01,2.64,0.25,3.97,3.68,2.81,3.47,2.66,2.17,-0.71,4.25,-1.69,4.38
1926-09-01,0.36,0.23,1.14,4.80,1.15,-3.39,-0.38,2.41,0.21,0.69,2.04,0.29
1926-10-01,-3.24,0.32,-1.24,-8.23,-3.63,-0.78,-4.58,-0.11,-2.29,-0.57,-2.63,-2.85
1926-11-01,2.53,0.31,5.21,-0.19,4.10,0.01,4.71,1.63,6.43,5.42,3.71,2.11


In [6]:
# choose sample period
df = df['1990':'2018']
df.head()

,Mkt-RF,RF,NoDur,Durbl,Manuf,Enrgy,HiTec,Telcm,Shops,Hlth,Utils,Other
Date,,,,,,,,,,,,
1990-01-01,-7.85,0.57,-9.43,-3.84,-6.29,-4.14,-1.28,-13.40,-6.24,-7.37,-5.35,-8.83
1990-02-01,1.11,0.57,-0.39,5.13,2.55,3.03,4.11,-0.34,1.92,-2.06,0.22,2.54
1990-03-01,1.83,0.64,4.54,2.90,3.91,-0.48,4.46,4.18,5.30,3.86,-0.75,-0.31
1990-04-01,-3.36,0.69,-0.27,-4.04,-2.74,-3.59,-2.00,-3.46,-1.91,0.29,-5.51,-3.47
1990-05-01,8.42,0.68,9.04,7.76,8.08,5.95,12.54,9.39,11.70,12.76,5.42,9.05


In [7]:
# %load return_binscatter.py
col = 'HiTec'
df[col + '-RF'] = df[col] - df['RF']
my_bins = pd.qcut(df['Mkt-RF'],10)
data = df[[col+'-RF', 'Mkt-RF']].groupby(my_bins).agg(['mean'])
data.plot.scatter(x=('Mkt-RF', 'mean'), y=(col+'-RF', 'mean'))

In [8]:
%%time
# run index model
import statsmodels.api as sm
import numpy as np

# make vector of ones (for constant in regression)
N = df.shape[0]
const = np.ones((N,1))

tab = pd.DataFrame(columns = ind.columns, index = ['alpha','beta','t-alpha','t-beta','var_e'])

# one time series regression for each industry
for col in ind.columns:
    
    # LHS: excess return
    y = df[col] - df['RF']
    
    # RHS: constant and market excess return
    X = df['Mkt-RF']
    X = sm.add_constant(X)
    
    # run regression
    res = sm.OLS(y,X).fit()
    
    # get alpha, beta, t-stats, and residual variance (called "scale" in statsmodels)
    tab[col] = np.hstack([res.params,res.tvalues,res.scale])

tab = tab.transpose()

from IPython.display import display
display(tab)

,alpha,beta,t-alpha,t-beta,var_e
NoDur,0.278110,0.623468,1.954900,18.741924,6.895658
Durbl,-0.254217,1.231483,-1.082844,22.432764,18.778766
Manuf,0.126990,1.002450,1.184158,39.975772,3.918391
Enrgy,0.119829,0.747885,0.500503,13.358991,19.529831
HiTec,0.064456,1.384320,0.336212,30.880311,12.522346
Telcm,-0.138439,0.951596,-0.835712,24.566558,9.349599
Shops,0.189198,0.888126,1.389320,27.890324,6.318563
Hlth,0.330713,0.722772,1.936074,18.095254,9.941444
Utils,0.330057,0.387852,1.721860,8.653022,12.519091
Other,-0.051766,1.078038,-0.430133,38.307418,4.934899


Wall time: 12.2 s


In [9]:
# Treynor-Black
# active portfolio 
port = tab['alpha'] / tab['var_e']

# passive portfolio
port.loc['Market'] = df['Mkt-RF'].mean() / df['Mkt-RF'].var() - (tab['beta'] * tab['alpha'] / tab['var_e']).sum()

In [11]:
# normalize so that the (active + passive) weights sum to one
port /= port.sum()
print (port)

NoDur     0.497422
Durbl    -0.166963
Manuf     0.399709
Enrgy     0.075674
HiTec     0.063483
Telcm    -0.182620
Shops     0.369302
Hlth      0.410285
Utils     0.325162
Other    -0.129375
Market   -0.662077
dtype: float64


In [12]:
# information ratio: sum of (alpha / st.dev.) in active portfolio
info = ((tab['alpha'] / np.sqrt(tab['var_e']))**2).sum()
print(info)

0.047801911811543064


In [13]:
# sharpe ratio of the market 
sharpe_m = df['Mkt-RF'].mean() / df['Mkt-RF'].std()
print(sharpe_m)

0.146023306752482


In [14]:
# total sharpe ratio
sq_sharpe = sharpe_m**2 + info**2
sharpe = np.sqrt(sq_sharpe)
print(sharpe)

0.15364839370383276
